「三模型 CE 值彙整比較 Excel 產生用程式」

In [4]:
import os
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment

# ===== 固定 IP 對照（來自你那張圖）=====
IP_MAPPING = [
    "麥寮-IP1",
    "崙背-IP2",
    "莿桐-IP3",
    "斗六(1)-IP5",
    "斗南-IP7",
    "虎尾-IP8",
    "土庫(2)-IP10",
    "褒忠-IP12",
    "東勢-IP13",
    "臺西(1)-IP14",
    "臺西(2)-IP15",
    "四湖-IP16",
    "口湖(1)-IP17",
    "水林(1)-IP18",
    "元長(3)-IP25",
    "大埤(1)-IP27",
    "大埤(2)-IP28",
]

MODEL_FOLDERS = ["SVM", "XGBoost", "XGBoost_AR"]
OUTPUT_FILE = "CE_Comparison.xlsx"
FONT_NAME = "標楷體"


def read_ce_value(index_path):
    try:
        wb = load_workbook(index_path, data_only=True)
        ws = wb.active
        return ws["C6"].value
    except Exception:
        return None


def collect_model_data(model_folder):
    """
    回傳 list，長度必為 68，順序依資料夾排序
    """
    values = []
    subfolders = sorted(
        [d for d in os.listdir(model_folder)
         if os.path.isdir(os.path.join(model_folder, d))]
    )

    for sub in subfolders:
        index_xlsx = os.path.join(model_folder, sub, "Index.xlsx")
        ce = read_ce_value(index_xlsx) if os.path.exists(index_xlsx) else None
        values.append(ce)

    return values


def generate_ce_excel():
    wb = Workbook()
    ws = wb.active
    ws.title = "CE Comparison"

    header_fill = PatternFill("solid", fgColor="BDD7EE")
    model_fill = PatternFill("solid", fgColor="E2EFDA")
    ip_fill = PatternFill("solid", fgColor="FFF2CC")

    header_font = Font(name=FONT_NAME, bold=True)
    normal_font = Font(name=FONT_NAME)
    center = Alignment(horizontal="center", vertical="center")

    # 收集三模型資料
    model_data = {m: collect_model_data(m) for m in MODEL_FOLDERS}

    # 表頭
    headers = ["地點 / 小時"] + MODEL_FOLDERS
    for col, text in enumerate(headers, start=1):
        cell = ws.cell(row=1, column=col, value=text)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center

    row = 2
    idx = 0

    for ip in IP_MAPPING:
        # IP 標題列
        ws.merge_cells(start_row=row, start_column=1, end_row=row, end_column=4)
        cell = ws.cell(row=row, column=1, value=ip)
        cell.font = header_font
        cell.fill = ip_fill
        cell.alignment = center
        row += 1

        # 4 小時
        for h in range(1, 5):
            ws.cell(row=row, column=1, value=f"T+{h}")
            ws.cell(row=row, column=1).font = normal_font
            ws.cell(row=row, column=1).alignment = center

            for col, model in enumerate(MODEL_FOLDERS, start=2):
                val = model_data[model][idx]
                cell = ws.cell(row=row, column=col, value=val)
                cell.font = normal_font
                cell.fill = model_fill
                cell.alignment = center

            idx += 1
            row += 1

    # 欄寬
    ws.column_dimensions["A"].width = 22
    for c in ["B", "C", "D"]:
        ws.column_dimensions[c].width = 18

    wb.save(OUTPUT_FILE)
    print(f"✅ 已正確產生 {OUTPUT_FILE}")





In [5]:

# 執行
generate_ce_excel()

✅ 已正確產生 CE_Comparison.xlsx
